# FP constructs

First functional programming language was LISP in late 1950s...

The concepts apply to any FP language (give or take particular language features)

(Scala, OCaml, Haskell, Erlang, Lisps...)

* Algebraic Data Types (ADT) and pattern matching
* Optional
* Higher order and polymorphic functions
* Immutable data structures
* Pattern matching


# Case classes and pattern matching

In [47]:
case class Person(name: String, lastName: String, age: Int, weight: Int, height_cm: Int)

defined class Person


In [52]:
val pedro = Person("Pedro", "Larroy", 39, 67, 174)
def bmi(person: Person): Double = person match {
    case Person(_, _, _, weight, height_cm) => (weight / (height_cm/100)^2) 
}
bmi(pedro)

pedro: Person = Person(Pedro,Larroy,39,67,174)
bmi: (person: Person)Double
res28: Double = 65.0


For making changes you have to create a copy

In [54]:
val mary = pedro.copy(name="Mary", lastName="Poppins")

mary: Person = Person(Mary,Poppins,39,67,174)


Case classes are an example of product types (the set of possible values are its cartessian product). 
Another example of product types are tuples.

One instance of ADTs are sum types

# Sum types

Sum types must be an instance of one of the types exclusively, ValidationError OR RuntimeError

In [83]:
sealed trait Error
case object ValidationError
case object RuntimeError

defined trait Error
defined object ValidationError
defined object RuntimeError


In [85]:
case class Tag(t: String, attrs: Object)
def parse(x: String): Either[Tag, Error] = { ??? }

defined class Tag
parse: (x: String)Either[Tag,Error]


In [94]:
def doSmth(x: String): Unit = parse(x) match {
    case Left(tag: Tag) => {???}
    case Right(error: Error) => {???}
}

<console>:33: warning: unreachable code
           case Right(error: Error) => {???}
                                        ^
doSmth: (x: String)Unit


# Optional


# Motivation: Python html parsing example

```
html = parse("""
    <p>
    <a href="https://www.w3schools.com">
    <img border="0" alt="W3Schools" src="logo_w3s.gif" width="100" height="100" />
    </a>
    </p>
    """)
# def get(self, k: _KT) -> Optional[_VT_co]
p = html.get("p")
if p:
    a = html.get("a")  
    if a:
        img = tag.get("img")
        [... do something ...]
```
We would like to chain operations / describe computation in a seamless way without specific handling of None

# Monads

![monad](https://miro.medium.com/max/700/0*fMijJbDb49jl5G1e)

# Scala Option monad

```
sealed abstract class Option[+A] ..
final case class Some[+A](value: A) extends Option[A] ...
case object None extends Option[Nothing]
...
final def map[B](f: (A) => B): Option[B]
final def flatMap[B](f: (A) => Option[B]): Option[B]
...
```

In [66]:
val maybe_s = Some("string value")
val maybe_sz = x.map(_.size)
def psize(x: Option[Int]): String = x match {
    case Some(x) => s"string length: $x"
    case None => "string is empty"
}
psize(maybe_sz)

maybe_s: Some[String] = Some(string value)
maybe_sz: Option[Int] = Some(12)
psize: (x: Option[Int])String
res37: String = string length: 12


In [81]:
val mx = Some(5)
mx.map(_*10).map(_*100).map(_*100)

mx: Some[Int] = Some(5)
res47: Option[Int] = Some(500000)


In [82]:
val mx = Some(5)
def times2(x: Int): Option[Int] = {
    Some(x*2)
}
mx.flatMap(times2)

mx: Some[Int] = Some(5)
times2: (x: Int)Option[Int]
res48: Option[Int] = Some(10)


In [69]:
val x = None : Option[Int]
x.map(_*10).map(_*100).map(_*100)

x: Option[Int] = None
res40: Option[Int] = None


So Option allows us to chain operations on types that might be None with safety and without if - else blocks
```
val html: Option[Tag] = parse("""
<p>
<a href="https://www.w3schools.com">
<img border="0" alt="W3Schools" src="logo_w3s.gif" width="100" height="100" />
</a>
</p>
""")
// def get(child: String): Option[Tag]
html.flatMap(_.get("p")).flatMap(_.get("a")).flatMap(_.get("img")).map(doSomething)
```

# Higher order and polymorphic functions

In [3]:
def partial1[A, B, C](a: A, f: (A,B) => C): B => C =  { ??? }

partial1: [A, B, C](a: A, f: (A, B) => C)B => C


In [6]:
partial1("a", (a: String, b: String) => a + b)

scala.NotImplementedError:  an implementation is missing

In [7]:
def partial1[A, B, C](a: A, f: (A,B) => C): B => C =  { 
    (b:B) => f(a, b)
}

partial1: [A, B, C](a: A, f: (A, B) => C)B => C


In [12]:
val f = partial1("The ", (a: String, b: String) => a + b)

f: String => String = $Lambda$2380/0x0000000840e72040@666f6485


In [13]:
f("light")

res6: String = The light


In [16]:
def curry[A,B,C](f: (A, B) => C): A => (B => C) = { ??? }

curry: [A, B, C](f: (A, B) => C)A => (B => C)


In [17]:
def curry[A,B,C](f: (A, B) => C): A => (B => C) = {
    (a: A) => partial1(a, f)
}

curry: [A, B, C](f: (A, B) => C)A => (B => C)


In [23]:
curry((a: String, b: String) => a + b)("The ")("light")

res12: String = The light


# Functional data structures
* As with referential transparency and pure functions, we want immutable data structures
* No need for locking -> improved concurrency
* Sequencing less important -> programs are more declarative
* Reduced side effects

![data structures](https://i.stack.imgur.com/2fjoA.png)

In [25]:
val xs = List(1,2,3)

xs: List[Int] = List(1, 2, 3)


In [30]:
val m = Map("John" -> 3, "Mary" -> 2)

m: scala.collection.immutable.Map[String,Int] = Map(John -> 3, Mary -> 2)


In [31]:
m("John")

res16: Int = 3


In [35]:
val all = m + ("Me" -> 4)

all: scala.collection.immutable.Map[String,Int] = Map(John -> 3, Mary -> 2, Me -> 4)


In [36]:
m

res19: scala.collection.immutable.Map[String,Int] = Map(John -> 3, Mary -> 2)


Let's check the definition of List from Scaladoc
```
traitMap[K, +V] extends Iterable[(K, V)] with collection.Map[K, V] with MapOps[K, V, Map, Map[K, V]] with MapFactoryDefaults[K, V, Map, Iterable]
```
The `+V` means the V type is covariant, so a `Map[String, Cat]` is also a `Map[String, Animal]` if  `Cat` extends `Animal`. A minus `-` sign indicates the reverse, also called contravariance.

```
class Foo[+A] // A covariant class
class Bar[-A] // A contravariant class
class Baz[A]  // An invariant class
```

# Pattern matching on Lists

In [37]:
val people = List("John", "Mary", "Peter")

people: List[String] = List(John, Mary, Peter)


In [39]:
def is_everybodyPresent[T](xs: List[T]): Boolean = xs match {
    case Nil => false
    case "John" :: "Mary" :: "Peter" :: Nil => true
    case _ => false
}

is_everybodyPresent: [T](xs: List[T])Boolean


In [40]:
is_everybodyPresent(people)

res20: Boolean = true


In [43]:
is_everybodyPresent("John" :: "Mary" :: Nil)

res23: Boolean = false


# Conclusion

FP constructs help us build reliable programs that use the type system to verify correctness at compile time.
FP constructs can create more scalable and parallelizable programs with better control of side effects and better resource utilization.
Programs can be easier to maintain and reason about.

For small programs, prototypes and scripts is usually not worth bothering about types, hence Python. Benefits compound beyond a certain scale for medium and large projects.